## A TSP Example

Find the closed tour that visits all ten cities and has the minimum total length. Distances between pairs of cities are given in the table below:

         ATL	 ORD	DEN	 IAH	 LAX	MIA	 JFK	SFO	 SEA	DCA
    ATL	 0	587	1212	701	1936	604	748	2139	2182	543
    ORD	587	0	920	940	1745	1188	713	1858	1737	597
    DEN1 212	920	0	879	831	1726	1631	949	1021	1494
    IAH	701	940	879	0	1379	968	1420	1645	1891	1220
    LAX	1936	1745	831	1379	0	2339	2451	347	959	2300
    MIA	604	1188	1726	968	2339	0	1092	2594	2734	923
    JFK	748	713	1631	1420	2451	1092	0	2571	2408	205
    SFO	2139	1858	949	1645	347	2594	2571	0	678	2442
    SEA	2182	1737	1021	1891	959	2734	2408	678	0	2329
    DCA	543	597	1494	1220	2300	923	205	2442	2329	0

In [1]:
# Define the problem data (cities and costs)
using JuMP, NamedArrays, Gurobi

cities = [:ATL, :ORD, :DEN, :IAH, :LAX, :MIA, :JFK, :SFO, :SEA, :DCA]

distances = [     0   587  1212   701  1936   604   748  2139  2182   543
                587     0   920   940  1745  1188   713  1858  1737   597
               1212   920     0   879   831  1726  1631   949  1021  1494
                701   940   879     0  1379   968  1420  1645  1891  1220
               1936  1745   831  1379     0  2339  2451   347   959  2300
                604  1188  1726   968  2339     0  1092  2594  2734   923
                748   713  1631  1420  2451  1092     0  2571  2408   205
               2139  1858   949  1645   347  2594  2571     0   678  2442
               2182  1737  1021  1891   959  2734  2408   678     0  2329
                543   597  1494  1220  2300   923   205  2442  2329     0  ]

c = NamedArray(distances,(cities,cities))
N = size(distances,1);

## Solve TSP using min-cost flow relaxation

In [3]:
# solve the simplest version (min-cost flow version; it's an assignment problem)

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

# LP relaxation
@variable(m, 0 <= x[cities,cities] <= 1)

# exactly one edge out of each node
@constraint(m, c1[j in cities], sum(x[i,j] for i in cities) == 1)

# esactly one edge into each node
@constraint(m, c2[i in cities], sum(x[i,j] for j in cities) == 1)

# no self-loops
@constraint(m, c3[i in cities], x[i,i] == 0)

# minimized total cost
@objective(m, Min, sum(x[i,j] * c[i,j] for i in cities, j in cities))

optimize!(m)

# pretty print the solution
xx = value.(x)
sol = NamedArray(zeros(Int,N,N), (cities,cities))

for i in cities
    for j in cities
        sol[i,j] = Int(xx[i,j])
    end
end

println(sol)
sleep(1)
display(getAllSubtours(sol))
;

Academic license - for non-commercial use only - expires 2022-07-06
10×10 Named Matrix{Int64}
A ╲ B │ :ATL  :ORD  :DEN  :IAH  :LAX  :MIA  :JFK  :SFO  :SEA  :DCA
──────┼───────────────────────────────────────────────────────────
:ATL  │    0     0     0     0     0     1     0     0     0     0
:ORD  │    0     0     0     1     0     0     0     0     0     0
:DEN  │    0     0     0     0     0     0     0     0     1     0
:IAH  │    0     1     0     0     0     0     0     0     0     0
:LAX  │    0     0     0     0     0     0     0     1     0     0
:MIA  │    1     0     0     0     0     0     0     0     0     0
:JFK  │    0     0     0     0     0     0     0     0     0     1
:SFO  │    0     0     0     0     1     0     0     0     0     0
:SEA  │    0     0     1     0     0     0     0     0     0     0
:DCA  │    0     0     0     0     0     0     1     0     0     0


5-element Vector{Any}:
 [:ATL, :MIA, :ATL]
 [:ORD, :IAH, :ORD]
 [:DEN, :SEA, :DEN]
 [:LAX, :SFO, :LAX]
 [:JFK, :DCA, :JFK]

In [6]:
xx

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:ATL, :ORD, :DEN, :IAH, :LAX, :MIA, :JFK, :SFO, :SEA, :DCA]
    Dimension 2, [:ATL, :ORD, :DEN, :IAH, :LAX, :MIA, :JFK, :SFO, :SEA, :DCA]
And data, a 10×10 Matrix{Float64}:
  0.0  -0.0  -0.0  -0.0   0.0   1.0  -0.0   0.0   0.0  -0.0
  0.0   0.0  -0.0  -0.0   0.0  -0.0   1.0   0.0   0.0   0.0
 -0.0   1.0   0.0  -0.0  -0.0   0.0   0.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0   0.0   1.0  -0.0  -0.0   0.0   0.0  -0.0
  0.0   0.0  -0.0  -0.0   0.0   0.0   0.0   1.0  -0.0   0.0
 -0.0  -0.0   0.0   1.0   0.0   0.0  -0.0   0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0   0.0  -0.0   0.0   0.0   0.0   1.0
  0.0   0.0  -0.0   0.0  -0.0   0.0   0.0   0.0   1.0   0.0
  0.0   0.0   1.0   0.0  -0.0   0.0   0.0  -0.0   0.0   0.0
  1.0  -0.0   0.0  -0.0   0.0  -0.0   0.0   0.0   0.0   0.0

## Solve TSP using adaptive subtour elimination

In [5]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

# must formulte as IP this time
@variable(m, x[cities,cities], Bin)

@constraint(m, c1[j in cities], sum( x[i,j] for i in cities ) == 1) 

@constraint(m, c2[i in cities], sum( x[i,j] for j in cities ) == 1)  

@constraint(m, c3[i in cities], x[i,i] == 0 )                  

@objective(m, Min, sum( x[i,j]*c[i,j] for i in cities, j in cities )) 

sols = []

# we'll run the heuristic 30 times and hope we get an optimal solution

for iters = 1:30
    optimize!(m)
    
    # total length of curent tour
    println("Tour length: ", objective_value(m))
    
    xx = value.(x) # save solution
    push!(sols,xx) # save solution
    
    subtours = getAllSubtours(xx) # get all the subtours
    display(subtours)
    sleep(1)
    
    # get the length of subtour list
    len = length(subtours)
    
    if len == 1
        # solution is just a single tour!
        println("SOLVED!")
        break
        
    else
        for subtour in subtours
            L = length(subtour)
            # add constraints that cut off each subtour in the list (add two for each subtour)
            @constraint(m, sum( x[subtour[k+1],subtour[k]] for k = 1:L-1 ) <= L-2)
            
            @constraint(m, sum( x[subtour[k],subtour[k+1]] for k = 1:L-1 ) <= L-2)
        end
    end
end

5-element Vector{Any}:
 [:ATL, :MIA, :ATL]
 [:ORD, :IAH, :ORD]
 [:DEN, :SEA, :DEN]
 [:LAX, :SFO, :LAX]
 [:JFK, :DCA, :JFK]

Academic license - for non-commercial use only - expires 2022-07-06
Tour length: 6234.0


3-element Vector{Any}:
 [:ATL, :IAH, :MIA, :ATL]
 [:ORD, :DCA, :JFK, :ORD]
 [:DEN, :LAX, :SFO, :SEA, :DEN]

Tour length: 6665.0


3-element Vector{Any}:
 [:ATL, :MIA, :DCA, :JFK, :ORD, :ATL]
 [:DEN, :IAH, :DEN]
 [:LAX, :SEA, :SFO, :LAX]

...

Tour length: 6991.0


3-element Vector{Any}:
 [:ATL, :ORD, :JFK, :DCA, :ATL]
 [:DEN, :SEA, :LAX, :SFO, :DEN]
 [:IAH, :MIA, :IAH]

Tour length: 7260.0


1-element Vector{Any}:
 [:ATL, :DCA, :JFK, :ORD, :DEN, :SEA, :SFO, :LAX, :IAH, :MIA, :ATL]

Tour length: 7378.0
SOLVED!


## Miller-Tucker-Zemlin formulation

### Get all subtours helper function

In [2]:
# HELPER FUNCTION: returns the cycle containing the city START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in cities
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = cities
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end

# HELPER FUNCTION FOR MAPPING AIRPORTS

data = [ 33.636700  -84.427863 
         41.977320  -87.908005
         39.861667 -104.673166
         29.645417  -95.278888
         33.942437 -118.408121
         25.795361  -80.290110
         40.639926  -73.778694
         37.618806 -122.375416
         47.449889 -122.311777
         38.851444  -77.037721 ]
lat = Dict(zip(cities,data[:,1]))
lon = Dict(zip(cities,data[:,2]))

Dict{Symbol, Float64} with 10 entries:
  :ATL => -84.4279
  :SFO => -122.375
  :DEN => -104.673
  :DCA => -77.0377
  :SEA => -122.312
  :ORD => -87.908
  :IAH => -95.2789
  :LAX => -118.408
  :JFK => -73.7787
  :MIA => -80.2901